### Requirements/Libraries


In [13]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib.util import guess_format
import owlrl

### Query local Playground dataset

In [3]:
#Loads KG
g = Graph()
g.parse("./data/playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

#Same but with loading a query from a file
print("Query results, Females (query from file):")
query_file = "./data/query.txt"
query = open(query_file, 'r').read()
    
qres = g.query(query)
    
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'
Query results, Females (query from file):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'


### Query local Nobel Prize dataset

In [4]:
#Loads KG
g = Graph()
g.parse("../files/nobelprize_kg.nt", format="nt")
  
print("Loaded '" + str(len(g)) + "' triples.")

#Query with prefixes 
nobelprize_query = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?name_laur WHERE { 
?laur rdf:type nobel:Laureate . 
?laur rdfs:label ?name_laur . 
?laur foaf:gender "female" . }
LIMIT 10
"""

qres = g.query(nobelprize_query)

print("Female laureates (printing 10):")
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("'%s'" % (str(row.name_laur))) #Same name as in sparql query SELECT variable


Loaded '85797' triples.
Female laureates (printing 10):
'Marie Curie, née Sklodowska'
'Maria Goeppert Mayer'
'Irène Joliot-Curie'
'Dorothy Crowfoot Hodgkin'
'Gerty Theresa Cori, née Radnitz'
'Rosalyn Yalow'
'Barbara McClintock'
'Rita Levi-Montalcini'
'Gertrude B. Elion'
'Christiane Nüsslein-Volhard'



### Query remote dataset via SPARQL Enpoint

In [5]:
#Remote service
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
LIMIT 10
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (limit to 10):")

for result in results["results"]["bindings"]:
    #Prints individual results 
    print(result["x"]["value"])
  

Stars co-starring Johnny Depp (limit to 10):

Abigail Breslin
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Aleksandr Kuznetsov
Alfred Molina
Alison Sudol


### Query World Cities Dataset

In [6]:
#Loads KG
g = Graph()
g.parse("./data/worldcities-free-100-task1.ttl", format="ttl")
  
    
print("Loaded '" + str(len(g)) + "' triples.")
    

print("Cities:")
    
qres = g.query(
"""SELECT ?city where {
   ?city rdf:type wco:City .      
}
LIMIT 10
""")
    
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is a City" % (str(row.city)))


Loaded '914' triples.
Cities:
http://www.semanticweb.org/ernesto/in3067-inm713/wco/ahmadabad is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bangalore is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bangkok is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/baoding is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/beijing is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bijie is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bogota is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/buenos_aires is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/cairo is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/cangzhou is a City


### Query World Cities Dataset: reasoning and CSV output

In [7]:
#Solution reasoning
def performReasoning(g, ontology_file):

    #We expand the graph with the inferred triples
    #We use owlrl library with OWL2 RL Semantics


    #We should load the ontology first
    g.parse(ontology_file,  format=guess_format(ontology_file)) #e.g., format=ttl

    print("Triples including ontology: '" + str(len(g)) + "'.")


    #We apply reasoning and expand the graph with new triples 
    owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(g)

    print("Triples after OWL 2 RL reasoning: '" + str(len(g)) + "'.")


#Solution query results to CSV
def performSPARQLQuery(g, file_query_out):

    qres = g.query(
        """SELECT DISTINCT ?country ?city ?pop WHERE {
            ?city rdf:type wco:City .
            ?city wco:isCapitalOf ?country .
            ?city wco:population ?pop .
            FILTER (xsd:integer(?pop) > 5000000)
    }
    ORDER BY DESC(?pop)
    """)


    print("%s capitals satisfying the query." % (str(len(qres))))
    print("Saving the following lines in the CSV file '%s'." % file_query_out)

    f_out = open(file_query_out,"w+")

    for row in qres:
        #Row is a list of matched RDF terms: URIs, literals or blank nodes
        line_str = '\"%s\",\"%s\",\"%s\"\n' % (row.country, row.city, row.pop)
        print(line_str)

        f_out.write(line_str)

    f_out.close()


In [14]:
#Solution Reasoning
performReasoning(g, "./data/ontology_worldcities.ttl")

Triples including ontology: '1027'.
Triples after OWL 2 RL reasoning: '3748'.


In [15]:
#Solution Query to CSV
performSPARQLQuery(g, "./data/query_result_world_cities.csv")

24 capitals satisfying the query.
Saving the following lines in the CSV file './data/query_result_world_cities.csv'.
"http://www.semanticweb.org/ernesto/in3067-inm713/wco/japan","http://www.semanticweb.org/ernesto/in3067-inm713/wco/tokyo","37977000"

"http://www.semanticweb.org/ernesto/in3067-inm713/wco/indonesia","http://www.semanticweb.org/ernesto/in3067-inm713/wco/jakarta","34540000"

"http://www.semanticweb.org/ernesto/in3067-inm713/wco/philippines","http://www.semanticweb.org/ernesto/in3067-inm713/wco/manila","23088000"

"http://www.semanticweb.org/ernesto/in3067-inm713/wco/korea,_south","http://www.semanticweb.org/ernesto/in3067-inm713/wco/seoul","21794000"

"http://www.semanticweb.org/ernesto/in3067-inm713/wco/mexico","http://www.semanticweb.org/ernesto/in3067-inm713/wco/mexico_city","20996000"

"http://www.semanticweb.org/ernesto/in3067-inm713/wco/china","http://www.semanticweb.org/ernesto/in3067-inm713/wco/beijing","19433000"

"http://www.semanticweb.org/ernesto/in3067-inm713/